# Academic Specific Code

## Install Libraries

In [ ]:
!pip install transformers torch
!pip install transformers torch scikit-learn
!pip install ipywidgets --upgrade
!pip install datasets --upgrade
!pip install pyarrow --upgrade
!pip install huggingface_hub
!pip install torch
!pip install transformers
!pip install scikit-learn
!pip install datasets
!pip install datasketch
!pip install transformers[torch] accelerate
!pip install ipywidgets
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!pip install requests
!pip install tiktoken
!pip install sentencepiece
!pip install --upgrade notebook ipywidgets
!pip install openai

## Import Libraries

In [ ]:
import ipywidgets as widgets
widgets.IntSlider()

import json
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sentencepiece import SentencePieceProcessor

print("SentencePiece is installed and ready to use.")

device = torch.device("cpu")
print(f"Using device: {device}")


## Configuration

In [ ]:
config = {
    "model_name": "UBC-NLP/araT5-base",  # Load AraT5 locally
    "fine_tune_model": "aubmindlab/bert-base-arabertv02",  # Model to fine-tune
    "threshold": 3,  # Minimum score for high-quality content
    "annotation_samples": 100,  # Total number of samples annotated with AraT5.
    "validation_samples": 30,  # Subset of annotation_samples reserved for validation
    "max_samples_to_fine_tune": 70,  # Maximum annotated samples used for fine-tuning
    "epochs": 5,  # Fine-tuning epochs
    "batch_size": 4,  # Lower the batch size to reduce memory usage
}

## Arabic Rubric Prompt

In [ ]:
arabic_prompt = """
فيما يلي مقتطف من صفحة ويب. قم بتقييم ما إذا كانت الصفحة ذات قيمة تعليمية عالية ويمكن أن تكون مفيدة في بيئة تعليمية لتدريس المستويات من المرحلة الابتدائية إلى المرحلة الإعدادية باستخدام نظام تقييم مكون من 5 نقاط تراكمية وفقًا للمعايير التالية:
أضف نقطة واحدة إذا كان المقتطف يقدم بعض المعلومات الأساسية ذات الصلة بالموضوعات التعليمية، حتى لو تضمن محتوى غير ذي صلة أو غير أكاديمي مثل الإعلانات والمواد الترويجية.
•	أضف نقطة أخرى إذا كان المقتطف يتناول بعض العناصر ذات الصلة بالتعليم ولكنه لا يتماشى بشكل وثيق مع المعايير التعليمية. قد يخلط بين المحتوى التعليمي وغير التعليمي، ويقدم نظرة عامة سطحية عن موضوعات قد تكون مفيدة، أو يعرض المعلومات بطريقة غير منظمة وأسلوب كتابة غير واضح.
•	امنح نقطة ثالثة إذا كان المقتطف مناسبًا للاستخدام التعليمي ويقدم مفاهيم رئيسية ذات صلة بالمناهج المدرسية. يكون المحتوى واضحًا ولكنه قد لا يكون شاملاً، أو قد يتضمن بعض المعلومات الزائدة. قد يشبه القسم التمهيدي لكتاب مدرسي أو درس تعليمي بسيط مناسب للتعلم ولكنه يحتوي على بعض القيود مثل معالجة مفاهيم معقدة جدًا لطلاب المرحلة الإعدادية.
•	امنح نقطة رابعة إذا كان المقتطف ذا صلة كبيرة ومفيدًا للأغراض التعليمية لمستوى لا يتجاوز المرحلة الإعدادية، مع أسلوب كتابة واضح ومتسق. يمكن أن يشبه فصلًا من كتاب مدرسي أو درسًا تعليميًا، حيث يقدم محتوى تعليميًا غنيًا، بما في ذلك التمارين والحلول، مع الحد الأدنى من المعلومات غير ذات الصلة، والمفاهيم ليست معقدة للغاية لطلاب هذه المرحلة. يكون المحتوى منظمًا ومركّزًا وقيمًا للتعلم المنهجي.
•	امنح نقطة خامسة إذا كان المقتطف ممتازًا في قيمته التعليمية ومناسبًا تمامًا للتدريس في المرحلة الابتدائية أو الإعدادية. يتبع المقتطف منطقًا تفصيليًا، وأسلوب الكتابة سهل الفهم، ويقدم رؤى عميقة وشاملة حول الموضوع دون أي محتوى غير تعليمي أو معقد.
المقتطف: <EXAMPLE>. بعد فحص المقتطف:
•	برر بإيجاز مجموع النقاط، بحد أقصى 100 كلمة.
•	اختتم بالنقاط الإجمالية بالتنسيق التالي: "التقييم التعليمي: <مجموع النقاط>".
"""

## Custom Dataset Class
standardizes the process of preparing text data for machine learning models by tokenizing text, truncating or padding sequences to a fixed length, and formatting inputs and labels into PyTorch tensors. This enables efficient batching and compatibility with PyTorch's DataLoader for training and evaluation.

In [ ]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

## Step 1: Load Dataset

In [ ]:
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return [{"text": item["text"], "metadata": item["metadata"]} for item in data]


## Step 2: Annotate Data Locally with AraT5

In [ ]:
def annotate_samples(samples, model_name):
    tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    annotated_data = []

    for sample in samples:
        text = sample["text"]
        prompt = arabic_prompt.format(text=text)

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(**inputs, max_length=100)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

        scores = extract_scores(result)
        annotated_data.append({"text": text, "scores": scores, "metadata": sample["metadata"]})

    return annotated_data

## Extract Scores

In [ ]:
def extract_scores(output):
    lines = output.split("\n")
    scores = {}
    for line in lines:
        if "ملاءمة" in line:
            scores["relevance"] = int(line.split(":")[-1].strip())
        elif "وضوح" in line:
            scores["clarity"] = int(line.split(":")[-1].strip())
        elif "عمق" in line:
            scores["depth"] = int(line.split(":")[-1].strip())
    total = sum(scores.values())
    scores["total"] = total
    return scores

## Step 3: Fine-Tune AraBERT

In [ ]:
def fine_tune_arabert(train_data, tokenizer, model):
    texts = [item["text"] for item in train_data]
    labels = [item["scores"]["total"] for item in train_data]

    dataset = CustomDataset(texts, labels, tokenizer)
    model.to(device)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=config["epochs"],
        per_device_train_batch_size=config["batch_size"],
        save_steps=10_000,
        save_total_limit=2,
        no_cuda=True,
        logging_dir="./logs",
        logging_steps=100,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    print("Starting fine-tuning...")
    trainer.train()
    print("Fine-tuning complete.")

## Step 4: Predict with Fine-Tuned AraBERT

In [ ]:
def predict_with_arabert(unlabeled_data, model, tokenizer):
    model.eval()
    model.to(device)
    predictions = []

    for sample in unlabeled_data:
        text = sample["text"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = logits.argmax(dim=-1).item()

        predictions.append({"text": text, "predicted_score": predicted_label, "metadata": sample["metadata"]})

    return predictions

## Step 5: Validate Model

In [ ]:
def validate_model(validation_data, model, tokenizer):
    model.to(device)
    true_labels = []
    predicted_labels = []

    for item in validation_data:
        text = item["text"]
        true_labels.append(item["scores"]["total"])
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = logits.argmax(dim=-1).item()
        predicted_labels.append(predicted_label)

    f1 = f1_score(true_labels, predicted_labels, average="macro")
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average="macro", zero_division=0)
    recall = recall_score(true_labels, predicted_labels, average="macro", zero_division=0)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    print(f"Validation F1 Score: {f1:.2f}")
    print(f"Validation Accuracy: {accuracy:.2f}")
    print(f"Validation Precision: {precision:.2f}")
    print(f"Validation Recall: {recall:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)

## Step 6: Filter Dataset

In [ ]:
def filter_dataset(annotated_data, threshold):
    return [
        doc for doc in annotated_data
        if doc["predicted_score"] >= threshold
    ]

## Main Pipeline

In [ ]:
def main_pipeline():
    # Step 1: Load the dataset from a specified JSON file
    dataset = load_dataset("/Users/ameeraattiah/Desktop/arabicweb24/jeje.json")
    print(f"Loaded {len(dataset)} samples.")

    # Step 2: Select a subset of the dataset for annotation
    sample_data = dataset[:config["annotation_samples"]]
    annotated_data = annotate_samples(sample_data, config["model_name"])
    print(f"Annotated {len(annotated_data)} samples.")

    # Step 3: Load the tokenizer and model for fine-tuning
    tokenizer = AutoTokenizer.from_pretrained(config["fine_tune_model"])
    model = AutoModelForSequenceClassification.from_pretrained(config["fine_tune_model"], num_labels=6)

    # Step 4: Fine-tune the model using the annotated data
    fine_tune_arabert(annotated_data, tokenizer, model)

    # Step 5: Use the fine-tuned model to predict the remaining dataset
    remaining_data = dataset[config["annotation_samples"]:]
    predictions = predict_with_arabert(remaining_data, model, tokenizer)
    print(f"Predicted {len(predictions)} samples with fine-tuned AraBERT.")

    # Step 6: Validate the fine-tuned model on a subset of the annotated data
    validation_data = annotated_data[:config["validation_samples"]]
    validate_model(validation_data, model, tokenizer)

    # Step 7: Filter the predictions to include only high-quality samples
    filtered_data = filter_dataset(predictions, config["threshold"])
    print(f"Filtered dataset contains {len(filtered_data)} high-quality samples.")

    # Step 8: Save the filtered data to a JSON file for future use
    with open("/Users/ameeraattiah/Desktop/arabicweb24/jeje-edu.json", "w", encoding="utf-8") as file:
        json.dump(filtered_data, file, ensure_ascii=False, indent=4)
    print("Filtered data saved.")



if __name__ == "__main__":
    main_pipeline()
    print("Running academic dataset processing...")

